# Data Cleaning
delete files in manually labeled data (**release-level-data** from replication kit https://zenodo.org/record/5675024#.Ya-B8fHML0o) that are not related to bug fixes

In [55]:
import os

for filename in os.listdir('./release-level-data'):
    if filename.endswith("bug_fixes.json"):
        continue
    os.remove('./release-level-data/' + filename)
print("data cleaning done")

data cleaning done


# Extract traditional smells
returns a dictionary of list, with key represents a traditional smell and value represents list of classes that contain the smell

In [30]:
import os
import subprocess
import configparser
import pandas as pd
import re
import shutil
import time
import sys

def extract_traditional_smells(project_name):
    print("extracting traditional smells from repository: " + project_name)
    subprocess.call(['java', '-jar', './DECOR_JAVA.jar', project_name, project_name, ""])

    # check if the result of detection is generated
    if os.path.isdir('../TEMP'):

        # dictionary of list, key represents a traditional smell and value represents list of classes that contain the smell
        smell_classes_dict = {}
        # iterate through each generated file
        result_files_path = "../TEMP/" + project_name
        for filename in os.listdir(result_files_path):

            # read the content of the file
            file_path = result_files_path + "/" + filename
            with open(file_path) as file:
                file_content = '[dummy_section]\n' + file.read()
            config = configparser.ConfigParser()
            config.read_string(file_content)

            # find all smelly test classes that contain traditional smell
            smelly_classes = []
            for key, value in config.items('dummy_section'):
                if re.match("^[0-9]+\.[0-9]+\.[a-zA-Z]+-\d$", key):
                    class_name = value.split(".")[-1]
                    smelly_classes.append(class_name)
            # map smell and classes containing the smell
            if smelly_classes:
                smell_name = filename.split(" ")[-1].split(".")[0]
                smell_classes_dict[smell_name] = smelly_classes

        # delete result of detection
        try:
            shutil.rmtree("../TEMP/")
        except OSError as e:
            print("Error: %s - %s." % (e.filename, e.strerror))
        finally:
            print("Traditional smells extracted for project: ", project_name)
            return smell_classes_dict

    else:    
        print("ERROR: extracting traditionial smells failed")
        return None

# Extract traditional smells test


In [114]:
smell_classes_dict = extract_traditional_smells('commons-bcel')
print(smell_classes_dict)

extracting traditional smells from repository: commons-bcel
Analysing commons-bcel...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.


java.util.MissingResourceException: Can't find resource for bundle util.multilingual.PtidejResourceBundle, key padl.kernel.IClass::ALREADY_IMPL
	at java.base/java.util.ResourceBundle.getObject(ResourceBundle.java:564)
	at java.base/java.util.ResourceBundle.getString(ResourceBundle.java:521)
	at util.multilingual.MultilingualManager.getString(MultilingualManager.java:65)
	at util.multilingual.MultilingualManager.getString(MultilingualManager.java:80)
	at padl.kernel.impl.AbstractClass.addImplementedInterface(AbstractClass.java:72)
	at padl.creator.javafile.eclipse.astVisitors.VisitorSecondParsing.manageSuperInterfaces(VisitorSecondParsing.java:315)
	at padl.creator.javafile.eclipse.astVisitors.VisitorSecondParsing.visit(VisitorSecondParsing.java:553)
	at org.eclipse.jdt.core.dom.TypeDeclaration.accept0(TypeDeclaration.java:453)
	at org.eclipse.jdt.core.dom.ASTNode.accept(ASTNode.java:2711)
	at org.eclipse.jdt.core.dom.ASTNode.acceptChildren(ASTNode.java:2782)
	at org.eclipse.jdt.core.do

FileNotFoundError: [Errno 2] No such file or directory: '../TEMP/commons-bcel'

# Extract traditional smells 2
returns a set of classes that contain a smell

In [20]:
import os
import subprocess
import configparser
import pandas as pd
import re
import shutil
import time
import sys

def extract_traditional_smells2(project_name):
    print("extracting traditional smells from repository: " + project_name)
    subprocess.call(['java', '-jar', './DECOR_JAVA.jar', project_name, project_name, ""])

    # check if the result of detection is generated
    if os.path.isdir('../TEMP'):

        # set of classes that contain the smell
        smell_classes_set = set()

        # iterate through each generated file
        result_files_path = "../TEMP/" + project_name
        for filename in os.listdir(result_files_path):

            # read the content of the file
            file_path = result_files_path + "/" + filename
            with open(file_path) as file:
                file_content = '[dummy_section]\n' + file.read()
            config = configparser.ConfigParser()
            config.read_string(file_content)

            # find all smelly test classes that contain traditional smell
            for key, value in config.items('dummy_section'):
                if re.match("^[0-9]+\.[0-9]+\.[a-zA-Z]+-\d$", key):
                    class_name = value.split(".")[-1]
                    smell_classes_set.add(class_name)

        # delete result of detection
        try:
            shutil.rmtree("../TEMP/")
        except OSError as e:
            print("Error: %s - %s." % (e.filename, e.strerror))
        finally:
            print("Traditional smells extracted for project: ", project_name)
            return smell_classes_set

    else:    
        print("ERROR: extracting traditionial smells failed")
        return None

# RQ1 - Fisher exact test

In [12]:
from git import Repo
import json
from scipy.stats import fisher_exact

# clone repo
# repo = Repo.clone_from('https://github.com/apache/ant-ivy', 'ant-ivy')
repo = Repo('ant-ivy')

# checkout to specific release
# print(repo.tags)
repo.git.checkout('1.4.1')

smelly_classes = extract_traditional_smells2('ant-ivy')
print(smelly_classes)

# number of classes with at least one fixing and with at least one anti-pattern 
w_fix_w_anti = 0
# number of classes with at least one fixing and without any anti-patterns
w_fix_wo_anti = 0
# number of classes without any fixing and with at least one anti-pattern 
wo_fix_w_anti = 0
# number of classes without any fixing and without any anti-patterns
wo_fix_wo_anti = 0

f = open('./release-level-data/ant-ivy-1.4.1_bug_fixes.json')
data = json.load(f)
for line in data:
    filename = line['file'].split("/")[-1].split(".")[0]
    if line['bug_fixes']:
        if filename in smelly_classes:
            w_fix_w_anti +=1
        else:
            w_fix_wo_anti += 1
    else:
        if filename in smelly_classes:
            wo_fix_w_anti +=1
        else:
            wo_fix_wo_anti += 1

f.close()

# computer fisher exact test
data = [[w_fix_w_anti, wo_fix_w_anti],
         [w_fix_wo_anti, wo_fix_wo_anti]]

oddsratio, pvalue = fisher_exact(data)
print('\n')
print('Fisher exact test odds ratio: {:.4f}'.format(oddsratio))
print('Fisher exact test p-value: {:.4f}'.format(pvalue))

extracting traditional smells from repository: ant-ivy
Analysing ant-ivy...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
Model built in 2148 ms.
Model contains 816 top-level entities.
2 solutions for AntiSingleton in ant-ivy in 7 ms.
util.repository.impl.FileRepositoryFactory has for running path: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.repository.impl.FileRepositoryFactory is creating a JarFileRepository for: util.repository.impl.JarFileRepository
util.repository.impl.JarFileRepository is the current repository on: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.lang.ConcreteReceiverGuard reports a runtime deprecation

Traditional smells extracted for project:  ant-ivy
{'PomModuleDescriptorParser', 'XmlModuleDescriptorParser', 'IvyNode', 'ResolverHelper', 'RepositoryResolver', 'DefaultArtifact', 'IvyCheck', 'IvyCacheTask', 'AntCallTrigger', 'XmlModuleDescriptorUpdater', 'IBiblioResolverTest', 'DefaultDependencyArtifactDescriptor', 'IvyBuildListTest', 'IvyCachePathTest', 'URLResolverTest', 'VfsResourceTest', 'IvyRepResolver', 'Message', 'DualResolverTest', 'ChainResolver', 'LatestRevisionStrategy', 'PomModuleDescriptorParserTest', 'BasicURLHandler', 'IvyConfigure', 'ResolveTest', 'AntBuildTrigger', 'SFTPRepository', 'SshCache', 'AbstractSshBasedRepository', 'CredentialsUtil', 'VfsURI', 'IvyInstall', 'IvyBuildNumber', 'VsftpRepository', 'HttpClientHandler', 'ConfigurationResolveReport', 'SortTest', 'TestPerformance', 'DefaultDependencyDescriptor', 'Main', 'IvyArtifactReport', 'AbstractResolver', 'Scp', 'IvyPatternHelper', 'AbstractResourceResolver', 'FileUtil', 'JarModuleFinder', 'IvyWebdavClientFactor

# RQ2 - Relation antipattern and fault-proneness

# Import de données

In [151]:
def getProjects():
    return { "ant-ivy": "https://github.com/apache/ant-ivy", "archiva": "https://github.com/apache/archiva", "calcite": "https://github.com/apache/calcite", "cayenne": "https://github.com/apache/cayenne", "commons-beanutils": "https://github.com/apache/commons-beanutils", "commons-codec": "https://github.com/apache/commons-codec", "commons-collections": "https://github.com/apache/commons-collections", "commons-compress": "https://github.com/apache/commons-compress", "commons-configuration": "https://github.com/apache/commons-configuration", "commons-dbcp": "https://github.com/apache/commons-dbcp", "commons-digester": "https://github.com/apache/commons-digester"}


In [152]:
def getRelease(isCorrespondance = False):
    if(isCorrespondance):
        return { "ant-ivy": [ "1.4.1", "2.0.0", "2.1.0", "2.2.0", "2.3.0", "2.4.0" ], "archiva": [ "1.0", "1.1", "1.2", "1.3", "2.0.0", "2.1.0", "2.2.0" ], "calcite": [ "1.8.0", "1.9.0", "1.10.0", "1.11.0", "1.12.0", "1.13.0", "1.14.0", "1.15.0" ], "cayenne": [ "3.0.0", "3.1.0" ], "commons-bcel": [ "5.2", "6.0", "6.1", "6.2" ], "commons-beanutils": [ "1.7.0", "1.8.0", "1.9.0" ], "commons-codec": [ "1.2", "1.3", "1.5", "1.6", "1.7", "1.8", "1.9", "1.10" ], "commons-collections": [ "1.0", "2.0", "2.1", "3.0", "3.1", "3.2", "3.3", "4.0", "4.1" ], "commons-compress": [ "1.9", "1.10", "1.11", "1.12", "1.13", "1.14", "1.15", "1.16" ], "commons-configuration": [ "1.4", "1.5", "1.6", "1.7", "1.8", "1.9", "1.10", "2.2" ], "commons-dbcp": [ "1.3", "1.4", "2.0", "2.1", "2.2.0", "2.3.0", "2.4.0", "2.5.0" ], "commons-digester": [ "1.4", "1.5", "1.6", "1.7", "1.8", "3.0", "3.1", "3.2" ] }
    else:
        return { "ant-ivy": [ "1.4.1", "2.0.0", "2.1.0", "2.2.0", "2.3.0", "2.4.0" ], "archiva": [ "archiva-1.0", "archiva-1.1", "archiva-1.2", "archiva-1.3", "archiva-2.0.0", "archiva-2.1.0", "archiva-2.2.0" ], "calcite": [ "calcite-1.8.0", "calcite-1.9.0", "calcite-1.10.0", "calcite-1.11.0", "calcite-1.12.0", "calcite-1.13.0", "calcite-1.14.0", "calcite-1.15.0" ], "cayenne": [ "STABLE-3.0", "STABLE-3.1" ], "commons-beanutils": [ "BEANUTILS_1_7_0", "BEANUTILS_1_8_0", "BEANUTILS_1_9_0" ], "commons-codec": [ "CODEC_1_2", "CODEC_1_3", "commons-codec-1.5", "1_6", "1.7", "1.8", "1.9", "1.10" ], "commons-collections": [ "collections-1.0", "collections-2.0", "collections-2.1", "collections-3.0", "collections-3.1", "collections-3.2", "collections-3.3", "collections-4.0", "collections-4.1" ], "commons-compress": [ "rel/1.9", "rel/1.10", "rel/1.11", "rel/1.12", "rel/1.13", "rel/1.14", "rel/1.15", "rel/1.16" ], "commons-configuration": [ "CONFIGURATION_1_4", "CONFIGURATION_1_5", "CONFIGURATION_1_6", "CONFIGURATION_1_7", "CONFIGURATION_1_8", "CONFIGURATION_1_9", "CONFIGURATION_1_10", "CONFIGURATION_2_2" ], "commons-dbcp": [ "DBCP_1_3", "DBCP_1_4", "DBCP_2_0", "DBCP_2_1", "DBCP_2_2_0", "DBCP_2_3_0", "commons-dbcp-2.4.0", "commons-dbcp-2.5.0" ], "commons-digester": [ "DIGESTER_1_4", "DIGESTER_1_5", "DIGESTER_1_6", "DIGESTER_1_7", "DIGESTER_1_8", "DIGESTER3_3_0", "DIGESTER3_3_1", "DIGESTER3_3_2" ] }

      
        
def getReleaseCorr(projectName, release):
    pos = getRelease()[projectName].index(release)
    return getRelease(True)[projectName][pos]
        

In [126]:
from git import Repo
import os

# clone repos
for project_name, git_url in getProjects().items():
    if not os.path.isdir("./"+project_name):
        print(git_url)
        print(project_name)
        repo = Repo.clone_from(git_url, project_name)
print("... Done!")

https://github.com/apache/ant-ivy
ant-ivy
https://github.com/apache/archiva
archiva
https://github.com/apache/calcite
calcite
https://github.com/apache/cayenne
cayenne
https://github.com/apache/commons-beanutils
commons-beanutils
https://github.com/apache/commons-codec
commons-codec
https://github.com/apache/commons-collections
commons-collections
https://github.com/apache/commons-compress
commons-compress
https://github.com/apache/commons-configuration
commons-configuration
https://github.com/apache/commons-dbcp
commons-dbcp
https://github.com/apache/commons-digester
commons-digester
... Done!


# Données automatiques

In [112]:
from git import Repo
from math import exp
import json


# clone repo
# repo = Repo.clone_from('https://github.com/apache/ant-ivy', projetName)

# Open result file
f = open('result.txt', 'w')
    
for project_name, git_url in getProjects().items():
    # projet name
    print('\n')
    print("===+> projet name", project_name)
    print('\n')
    repo = Repo(project_name)

    # Seuil en dessous duquel il n'y a pas correlation
    seuil = 0.75

    # Write project name in file
    f.write(project_name)
    f.write('\n')

    # checkout to specific release
    # print('\n')
    # print('-- TAGS --')
    # print(repo.tags)

    # dictionnaire des correlation de chaque smell avec les classes
    smell_correlation_dict = {}

    tags = getRelease()[project_name]
    for tag in tags:
        print('\n')
        print(tag)

        repo.git.checkout(tag)

        # smelly_classes = extract_traditional_smells2('ant-ivy')
        # print('\n')
        # print('-- Smelly classes --')
        # print(smelly_classes)

        smell_classes_dict = extract_traditional_smells(project_name)
        # print('-- Smell in classes --')
        # print(smell_classes_dict)
        # print('\n')


        for smell, classes in smell_classes_dict.items():
            size = len(classes)
            size_exp = exp(size)
            # print("size : ",size)
            # print("size_exp", size_exp)
            correlation_value = (size_exp)/(1+size_exp)

            if correlation_value > seuil:
                # print(smell, " : ", correlation_value)
                smell_correlation_dict[smell] = smell_correlation_dict.get(smell, 0) +1



    f.write(str(smell_correlation_dict))
    f.write('\n')
    print(smell_correlation_dict)

f.close()
print("... Done!")



===+> projet name commons-dbcp




DBCP_1_3
extracting traditional smells from repository: commons-dbcp
Analysing commons-dbcp...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
Model built in 21996 ms.
Model contains 325 top-level entities.
0 solutions for AntiSingleton in commons-dbcp in 4 ms.
util.repository.impl.FileRepositoryFactory has for running path: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.repository.impl.FileRepositoryFactory is creating a JarFileRepository for: util.repository.impl.JarFileRepository
util.repository.impl.JarFileRepository is the current repository on: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar

Analysing commons-dbcp...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
Model built in 32644 ms.
Model contains 330 top-level entities.
0 solutions for AntiSingleton in commons-dbcp in 6 ms.
util.repository.impl.FileRepositoryFactory has for running path: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.repository.impl.FileRepositoryFactory is creating a JarFileRepository for: util.repository.impl.JarFileRepository
util.repository.impl.JarFileRepository is the current repository on: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "pom.metrics.repository.ACA

extracting traditional smells from repository: commons-dbcp
Analysing commons-dbcp...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
Model built in 17028 ms.
Model contains 354 top-level entities.
0 solutions for AntiSingleton in commons-dbcp in 4 ms.
util.repository.impl.FileRepositoryFactory has for running path: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.repository.impl.FileRepositoryFactory is creating a JarFileRepository for: util.repository.impl.JarFileRepository
util.repository.impl.JarFileRepository is the current repository on: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.lang.ConcreteReceiverGuard reports a run

extracting traditional smells from repository: commons-dbcp
Analysing commons-dbcp...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
Model built in 17428 ms.
Model contains 366 top-level entities.
0 solutions for AntiSingleton in commons-dbcp in 4 ms.
util.repository.impl.FileRepositoryFactory has for running path: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.repository.impl.FileRepositoryFactory is creating a JarFileRepository for: util.repository.impl.JarFileRepository
util.repository.impl.JarFileRepository is the current repository on: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.lang.ConcreteReceiverGuard reports a run

extracting traditional smells from repository: commons-dbcp
Analysing commons-dbcp...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
Model built in 18577 ms.
Model contains 369 top-level entities.
0 solutions for AntiSingleton in commons-dbcp in 4 ms.
util.repository.impl.FileRepositoryFactory has for running path: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.repository.impl.FileRepositoryFactory is creating a JarFileRepository for: util.repository.impl.JarFileRepository
util.repository.impl.JarFileRepository is the current repository on: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.lang.ConcreteReceiverGuard reports a run

extracting traditional smells from repository: commons-dbcp
Analysing commons-dbcp...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
Model built in 19218 ms.
Model contains 370 top-level entities.
0 solutions for AntiSingleton in commons-dbcp in 4 ms.
util.repository.impl.FileRepositoryFactory has for running path: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.repository.impl.FileRepositoryFactory is creating a JarFileRepository for: util.repository.impl.JarFileRepository
util.repository.impl.JarFileRepository is the current repository on: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.lang.ConcreteReceiverGuard reports a run

extracting traditional smells from repository: commons-dbcp
Analysing commons-dbcp...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.


KeyboardInterrupt: 

# Données manuelles et openSSZ

In [150]:
from git import Repo



# Seuil en dessous duquel il n'y a pas correlation
seuil = 0.75

# Open result file
ff = open('result2.txt', 'w')

for project_name, git_url in getProjects().items():

    # dictionnaire des correlation de chaque smell avec les classes
    smell_correlation_dict = {}
    
    # projet name
    print('\n')
    print("===+> projet name", project_name)
    print('\n')
    repo = Repo(project_name)

    
    # Write project name in file
    ff.write(project_name)
    ff.write('\n')
    
    # checkout to specific release
    tags = getRelease()[project_name]
    for tag in tags:
        
        # dictionnaire des smells
        manu_smell_dict = {}
        
        tagCorr = getReleaseCorr(project_name, tag)
        print('\n')
        print('tag : ', tag)
        print('tag corr : ', tagCorr)
        
        # checkout to specific release
        repo.git.checkout(tag)

        smelly_classes = extract_traditional_smells(project_name)



        f = open('./release-level-data/'+project_name+'-'+tagCorr+'_bug_fixes.json')
        data = json.load(f)
        for line in data:
            filename = line['file'].split("/")[-1].split(".")[0]
            if line['bug_fixes']:
                #print(filename)

                for smell, classes in smelly_classes.items():
                    if filename in classes:
                        manu_smell_dict.setdefault(smell, []).append(filename)


        print(manu_smell_dict)

        for smell, classes in manu_smell_dict.items():
            size = len(classes)
            size_exp = exp(size)
            print("size : ",size)
            print("size_exp", size_exp)
            correlation_value = (size_exp)/(1+size_exp)

            if correlation_value > seuil:
                print(smell, " : ", correlation_value)
                smell_correlation_dict[smell] = smell_correlation_dict.get(smell, 0) +1
        f.close()
        
    print('\n')
    print('\n')
    print('--- Resultats ---')
    ff.write(str(smell_correlation_dict))
    ff.write('\n')
    print(smell_correlation_dict)
ff.close()
print("... Done!")



===+> projet name ant-ivy




tag :  1.4.1
tag corr :  1.4.1
extracting traditional smells from repository: ant-ivy
Analysing ant-ivy...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
Model built in 2090 ms.
Model contains 816 top-level entities.
util.io.ProxyDisk reports that "../Temp/ant-ivy/DetectionResults in ant-ivy for AntiSingleton.ini already exists ("/Users/aurelikama/Documents/Projet/patron/projetFinal/patron/../Temp/ant-ivy/DetectionResults in ant-ivy for AntiSingleton.ini")
2 solutions for AntiSingleton in ant-ivy in 4 ms.
util.repository.impl.FileRepositoryFactory has for running path: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.repository.im

Traditional smells extracted for project:  ant-ivy
{'ComplexClass': ['Configurator', 'XmlIvyConfigurationParser', 'ChainResolver', 'AbstractModuleDescriptorParser', 'DefaultDependencyDescriptor', 'BasicResolver', 'IvyResolve', 'Message', 'IvyPostResolveTask', 'IvyTask', 'XmlModuleDescriptorParser', 'Ivy', 'RepositoryResolver', 'IvyReport', 'IvyNode', 'HttpClientHandler'], 'LongMethod': ['IvyArtifactProperty', 'IvyCacheTask', 'DefaultDependencyDescriptor', 'Message', 'IvyPostResolveTask', 'IvyTask', 'AntBuildTrigger', 'AntCallTrigger', 'RepositoryResolver', 'ModuleRevisionId', 'IvyReport', 'VfsRepository', 'ApacheURLLister', 'ModuleDescriptorSorter', 'IvyConfigure', 'HttpClientHandler'], 'LongParameterList': ['DefaultDependencyDescriptor', 'IvyTask', 'Ivy', 'RepositoryResolver', 'ModuleRevisionId', 'IvyReport', 'IvyNode']}
size :  16
size_exp 8886110.520507872
ComplexClass  :  0.9999998874648379
size :  16
size_exp 8886110.520507872
LongMethod  :  0.9999998874648379
size :  7
size_exp 1

0 solutions for MessageChains in ant-ivy in 5 ms.
0 solutions for RefusedParentBequest in ant-ivy in 16 ms.
0 solutions for SpaghettiCode in ant-ivy in 7 ms.
0 solutions for SpeculativeGenerality in ant-ivy in 2 ms.
0 solutions for SwissArmyKnife in ant-ivy in 0 ms.
0 solutions for TraditionBreaker in ant-ivy in 4 ms.
Traditional smells extracted for project:  ant-ivy
{'LongMethod': ['Ivy', 'AntBuildTrigger', 'AntCallTrigger', 'IvyCacheFileset', 'IvyCacheTask', 'IvyReport', 'IvyRepositoryReport', 'XMLHelper', 'XmlSettingsParser', 'SortEngine', 'XmlReportWriter', 'IBiblioResolver', 'RepositoryResolver', 'VersionRangeMatcher', 'BasicURLHandler', 'HttpClientHandler', 'ModuleRevisionId', 'SFTPRepository', 'AbstractSshBasedRepository', 'URLRepository', 'PomReader', 'PomModuleDescriptorBuilder', 'Main'], 'ComplexClass': ['Ivy', 'IvyBuildList', 'IvyPublish', 'IvyReport', 'IvyRepositoryReport', 'IvyResolve', 'IvyPatternHelper', 'XMLHelper', 'DefaultRepositoryCacheManager', 'PublishEngine', 'Iv

97 solutions for LongMethod in ant-ivy in 11 ms.
47 solutions for LongParameterList in ant-ivy in 6 ms.
0 solutions for ManyFieldAttributesButNotComplex in ant-ivy in 3 ms.
0 solutions for MessageChains in ant-ivy in 5 ms.
0 solutions for RefusedParentBequest in ant-ivy in 19 ms.
0 solutions for SpaghettiCode in ant-ivy in 7 ms.
0 solutions for SpeculativeGenerality in ant-ivy in 2 ms.
0 solutions for SwissArmyKnife in ant-ivy in 0 ms.
0 solutions for TraditionBreaker in ant-ivy in 4 ms.
Traditional smells extracted for project:  ant-ivy
{'LongMethod': ['RepositoryResolver', 'Main', 'Ivy', 'AntBuildTrigger', 'AntCallTrigger', 'IvyAntSettings', 'IvyCacheFileset', 'IvyCacheTask', 'IvyInfo', 'IvyListModules', 'IvyReport', 'IvyRepositoryReport', 'XMLHelper', 'IvySettings', 'XmlSettingsParser', 'SortEngine', 'LatestCompatibleConflictManager', 'XmlReportWriter', 'AbstractResolver', 'FileSystemResolver', 'IBiblioResolver', 'VersionRangeMatcher', 'CommandLineParser', 'ApacheURLLister', 'BasicU

48 solutions for LongParameterList in ant-ivy in 5 ms.
0 solutions for ManyFieldAttributesButNotComplex in ant-ivy in 4 ms.
0 solutions for MessageChains in ant-ivy in 5 ms.
0 solutions for RefusedParentBequest in ant-ivy in 19 ms.
0 solutions for SpaghettiCode in ant-ivy in 14 ms.
0 solutions for SpeculativeGenerality in ant-ivy in 2 ms.
0 solutions for SwissArmyKnife in ant-ivy in 1 ms.
0 solutions for TraditionBreaker in ant-ivy in 4 ms.
Traditional smells extracted for project:  ant-ivy
{'LongMethod': ['Ivy', 'IvyCacheFileset', 'IvyInfo', 'IvyListModules', 'IvyRepositoryReport', 'CredentialsUtil', 'XMLHelper', 'SortEngine', 'XmlReportWriter', 'ChainResolver', 'IBiblioResolver', 'VersionRangeMatcher', 'CommandLineParser', 'HttpClientHandler', 'PomReader', 'XmlModuleDescriptorUpdater', 'SFTPRepository', 'AbstractSshBasedRepository', 'URLRepository', 'ResolverHelper', 'RepositoryResolver', 'BasicURLHandler', 'IvyAuthenticator', 'IvyArtifactReport', 'ApacheURLLister'], 'ComplexClass': 

...

extracting traditional smells from repository: commons-digester
Analysing commons-digester...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
Model built in 21628 ms.
Model contains 339 top-level entities.
10 solutions for AntiSingleton in commons-digester in 6 ms.
util.repository.impl.FileRepositoryFactory has for running path: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.repository.impl.FileRepositoryFactory is creating a JarFileRepository for: util.repository.impl.JarFileRepository
util.repository.impl.JarFileRepository is the current repository on: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.lang.ConcreteReceiverGuard 

extracting traditional smells from repository: commons-digester
Analysing commons-digester...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
Model built in 42995 ms.
Model contains 348 top-level entities.
10 solutions for AntiSingleton in commons-digester in 6 ms.
util.repository.impl.FileRepositoryFactory has for running path: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.repository.impl.FileRepositoryFactory is creating a JarFileRepository for: util.repository.impl.JarFileRepository
util.repository.impl.JarFileRepository is the current repository on: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.lang.ConcreteReceiverGuard 

extracting traditional smells from repository: commons-digester
Analysing commons-digester...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
Model built in 1571 ms.
Model contains 583 top-level entities.
0 solutions for AntiSingleton in commons-digester in 4 ms.
util.repository.impl.FileRepositoryFactory has for running path: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.repository.impl.FileRepositoryFactory is creating a JarFileRepository for: util.repository.impl.JarFileRepository
util.repository.impl.JarFileRepository is the current repository on: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.lang.ConcreteReceiverGuard re

extracting traditional smells from repository: commons-digester
Analysing commons-digester...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
Model built in 1443 ms.
Model contains 590 top-level entities.
0 solutions for AntiSingleton in commons-digester in 4 ms.
util.repository.impl.FileRepositoryFactory has for running path: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.repository.impl.FileRepositoryFactory is creating a JarFileRepository for: util.repository.impl.JarFileRepository
util.repository.impl.JarFileRepository is the current repository on: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.lang.ConcreteReceiverGuard re

extracting traditional smells from repository: commons-digester
Analysing commons-digester...
I am here at analyseCodeLevelModelFromJavaSourceFilesEclipse class 521
util.lang.ConcreteReceiverGuard reports a runtime deprecation: calling method "padl.kernel.impl.CodeLevelModel.create()" from class "sad.detection.generators.SmellDetectionHelper"
Please use the methods in "padl.generator.helper.ModelGenerator" to obtain code-level models.
Model built in 1568 ms.
Model contains 609 top-level entities.
0 solutions for AntiSingleton in commons-digester in 4 ms.
util.repository.impl.FileRepositoryFactory has for running path: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.repository.impl.FileRepositoryFactory is creating a JarFileRepository for: util.repository.impl.JarFileRepository
util.repository.impl.JarFileRepository is the current repository on: /Users/aurelikama/Documents/Projet/patron/projetFinal/patron/DECOR_JAVA.jar
util.lang.ConcreteReceiverGuard re

# Nettoyage de données

In [153]:
import os
import shutil

# clone repos
for project_name, git_url in getProjects().items():
    if os.path.isdir("./"+project_name):
        print(project_name)
        shutil.rmtree("./"+project_name)
print("... Done!")

ant-ivy
archiva
calcite
cayenne
commons-beanutils
commons-codec
commons-collections
commons-compress
commons-configuration
commons-dbcp
commons-digester
... Done!
